In [3]:
import json
import pandas as pd
from datetime import datetime
import copy
import os
from cache import Cache
from preprocessing_data import vec2string, lemmatizer, stemming, remove_stopwords, tokenization, remove_punct

cache=Cache()

C:\Users\auges\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
liste =[]
i=1
while os.path.exists(f'../Database/Database/diversified_{i}.json'):
    with open(f'../Database/Database/diversified_{i}.json') as file:
        liste += json.load(file)
    i+=1

In [5]:
for elem in liste:
    elem['date'] = ",".join([str(e) for e in elem['date']])

In [6]:
df = pd.DataFrame(liste)

In [ ]:
df.head()

In [25]:
def preprocessing(x):
    
    return (
    vec2string(
        lemmatizer(
            stemming(
                remove_stopwords(
                    tokenization(
                        remove_punct(x).lower()
                    )
                )
            )
        )
    ))



In [26]:
def apply_preprocessing(df,nom_colonne):
    
    df[nom_colonne] = df[nom_colonne].apply(lambda x : preprocessing(x))
    return(df)

In [58]:
if cache.check('clean_df_complete'):
    clean_df_complete = cache.load('clean_df_complete')
else : 
    clean_df_complete = apply_preprocessing(df,'content')
    clean_df_complete = apply_preprocessing(clean_df_complete,'title')
    cache.save(clean_df_complete,'clean_df_complete')

## Word2Vec

In [32]:
import pandas as pd


import tqdm

import nltk
from nltk.corpus import wordnet
import gensim
import pickle

import sys
import os
import networkx as nx
import numpy as np

from gensim.models.word2vec import Word2Vec

WORD2VEC_DIMENSION = 32

In [184]:
if cache.check('content_w2v'):
    content_w2v = cache.load('content_w2v')
else : 
    content_w2v = Word2Vec(clean_df_complete['content'], size=WORD2VEC_DIMENSION, window=5, min_count=4, workers=10)
    content_w2v.train(clean_df_complete['content'], epochs=500, total_examples=content_w2v.corpus_count)
    cache.save(content_w2v,'content_w2v')

In [35]:
def compute_embedding(data, w2v):
    vectors = []
    for c in data:
        vec = np.zeros(WORD2VEC_DIMENSION)
        nb = 0
        for w in c:
            if w in w2v.wv:
                nb += 1
                vec += w2v.wv[w]

        if nb:
            vec /= nb
        vectors.append(vec)
        
    return vectors

In [60]:
if cache.check('title_w2v'):
    title_w2v = cache.save('title_w2v')
else:
    title_w2v = Word2Vec(clean_df_complete['title'], size=WORD2VEC_DIMENSION, window=5, min_count=4, workers=10)
    title_w2v.train(clean_df_complete['title'], epochs=500, total_examples=title_w2v.corpus_count)
    cache.save(title_w2v,'title_w2v')

(361835047, 2020961000)

In [61]:
cache.save(description_w2v,'description_w2v')
cache.save(title_w2v,'title_w2v')

In [183]:
cache.save(content_w2v,'content_w2v')

In [62]:
feature_description_w2v = compute_embedding(clean_df_complete['description'],description_w2v)
feature_title_w2v = compute_embedding(clean_df_complete['title'],title_w2v)

In [185]:
feature_content_w2v = compute_embedding(clean_df_complete['content'],content_w2v)

In [81]:
clean_df_complete['thematic'].value_counts()

culture             13706
politique           13418
sport               12063
planete             11696
economie             8435
actualite-medias     7192
societe              6701
sante                6308
campus               5984
sciences             5484
international        5414
Name: thematic, dtype: int64

# labels

In [82]:
def filter_df(df,column_name,limit_value):
    
    df2 = pd.DataFrame.copy(df.loc[df[column_name].isin(df[column_name].value_counts().loc[df[column_name].value_counts() >= limit_value].index)])
    theme_names = df[column_name].value_counts().loc[df[column_name].value_counts() >= limit_value].index
    return (df2,theme_names)
    
df2,theme_names = filter_df(clean_df_complete,'thematic',0)

In [83]:
dic_theme_names = {}
reverse_dic_theme_names = {}
for i,item in enumerate(theme_names):
    dic_theme_names[item]=i
    reverse_dic_theme_names[i]=item

In [84]:
dic_theme_names

{'culture': 0,
 'politique': 1,
 'sport': 2,
 'planete': 3,
 'economie': 4,
 'actualite-medias': 5,
 'societe': 6,
 'sante': 7,
 'campus': 8,
 'sciences': 9,
 'international': 10}

In [118]:
def f1(nom):
    return dic_theme_names[nom]

labels = pd.DataFrame(df2['thematic'].apply(lambda x : f1(x))).rename(index=str, columns={"thematic": "thematic_value"})

In [119]:
labels['article_id'] = [i for i in range(len(labels))]
labels['thematic'] = [df2['thematic'][i] for i in range(len(labels))]

In [121]:
labels = labels[['article_id','thematic','thematic_value']]

In [125]:
labels.shape

(96401, 3)

# saving

In [126]:
cache.save(feature_description_w2v,'feature_description_w2v_96k')
cache.save(feature_title_w2v,'feature_title_w2v_96k')
cache.save(labels,'labels_96k')

In [187]:
cache.save(feature_content_w2v,'feature_content_w2v_96k')

# TFIDF LSA

In [133]:
import os.path
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

from gensim.models import TfidfModel
from gensim.similarities import MatrixSimilarity

import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

import pandas as pd

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize


import nltk
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import nltk.internals
from gensim import corpora, models


from numpy import dot
from numpy.linalg import norm

from collections import defaultdict

C:\Users\auges\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [134]:
corpus = clean_df_complete['title'].values
dictionary = corpora.Dictionary([d.split() for d in corpus])
corpus_gensim = [dictionary.doc2bow(doc.split()) for doc in corpus]   
tfidf = TfidfModel(corpus_gensim)
corpus_tfidf = tfidf[corpus_gensim]
lsi = LsiModel(corpus_tfidf, id2word=dictionary, num_topics=5)

In [135]:
def text2Vec_LSA(text):
    vec = dictionary.doc2bow(text.lower().split())   
    vec_lsi = lsi[vec]
    return np.array([a[1] for a in vec_lsi])

def LSA_dataframe(data, column, n_topics):
    LSA = pd.DataFrame(index=data.index, columns=['dim_' + str(i) for i in range(1,n_topics + 1)])
    dt = data[column].apply(lambda x: text2Vec_LSA(x))
    return dt

In [169]:
feature_title_lsa = clean_df_complete['title'].apply(lambda x : text2Vec_LSA(x))

In [172]:
corpus = clean_df_complete['content'].values
dictionary = corpora.Dictionary([d.split() for d in corpus])
corpus_gensim = [dictionary.doc2bow(doc.split()) for doc in corpus]   
tfidf = TfidfModel(corpus_gensim)
corpus_tfidf = tfidf[corpus_gensim]
lsi = LsiModel(corpus_tfidf, id2word=dictionary, num_topics=5)

In [173]:
feature_content_lsa = clean_df_complete['content'].apply(lambda x : text2Vec_LSA(x))

In [188]:
corpus = clean_df_complete['description'].values
dictionary = corpora.Dictionary([d.split() for d in corpus])
corpus_gensim = [dictionary.doc2bow(doc.split()) for doc in corpus]   
tfidf = TfidfModel(corpus_gensim)
corpus_tfidf = tfidf[corpus_gensim]
lsi = LsiModel(corpus_tfidf, id2word=dictionary, num_topics=5)

In [189]:
feature_description_lsa = clean_df_complete['description'].apply(lambda x : text2Vec_LSA(x))

In [174]:
cache.save(feature_content_lsa,'feature_content_lsa_96k')
cache.save(feature_title_lsa,'feature_title_lsa_96k')

In [190]:
cache.save(feature_description_lsa,'feature_description_lsa_96k')

# Using keras NN

In [176]:
rd.seed(a = 10000)

In [180]:
labels  = cache.load('labels_96k')
content_lsa = cache.load('feature_content_lsa_96k')
title_lsa = cache.load('feature_title_lsa_96k')
content_w2v = cache.load('feature_content_w2v_96k')
title_w2v = cache.load('feature_title_w2v_96k')

Y = labels['thematic_value']

FileNotFoundError: [Errno 2] No such file or directory: 'cache_dir/feature_content_w2v_96k'